# Standard packages

In [6]:
import os
import pandas as pd
import time

# Specific installs

IPKnot need ViennaRNA

In [1]:
method_name = "IPknot"

In [2]:
!rm *.deb
!wget https://www.tbi.univie.ac.at/RNA/download/ubuntu/ubuntu_18_04/viennarna_2.5.0-1_amd64.deb
!wget https://www.tbi.univie.ac.at/RNA/download/ubuntu/ubuntu_18_04/viennarna-dev_2.5.0-1_amd64.deb
!apt-get -q install ./viennarna_2.5.0-1_amd64.deb ./viennarna-dev_2.5.0-1_amd64.deb

rm: cannot remove '*.deb': No such file or directory
--2024-11-08 19:06:24--  https://www.tbi.univie.ac.at/RNA/download/ubuntu/ubuntu_18_04/viennarna_2.5.0-1_amd64.deb
Resolving www.tbi.univie.ac.at (www.tbi.univie.ac.at)... 131.130.44.61
Connecting to www.tbi.univie.ac.at (www.tbi.univie.ac.at)|131.130.44.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3448392 (3.3M) [application/x-debian-package]
Saving to: ‘viennarna_2.5.0-1_amd64.deb’

viennarna_2.5.0-1_a 100%[===================>]   3.29M  3.57MB/s    in 0.9s    

2024-11-08 19:06:26 (3.57 MB/s) - ‘viennarna_2.5.0-1_amd64.deb’ saved [3448392/3448392]

--2024-11-08 19:06:26--  https://www.tbi.univie.ac.at/RNA/download/ubuntu/ubuntu_18_04/viennarna-dev_2.5.0-1_amd64.deb
Resolving www.tbi.univie.ac.at (www.tbi.univie.ac.at)... 131.130.44.61
Connecting to www.tbi.univie.ac.at (www.tbi.univie.ac.at)|131.130.44.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 43601504 (42M) [appli

Also needs GLPK

In [3]:
!apt-get -q install glpk-utils libglpk-dev

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libamd2 libbtf1 libcamd2 libccolamd2 libcholmod3 libcolamd2 libcxsparse3 libglpk40 libgmp-dev
  libgmpxx4ldbl libgraphblas-dev libgraphblas6 libklu1 libldl2 libmetis5 libmongoose2 librbio2
  libsliplu1 libspqr2 libsuitesparse-dev libsuitesparseconfig5 libumfpack5
Suggested packages:
  libiodbc2-dev gmp-doc libgmp10-doc libmpfr-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libbtf1 libcamd2 libccolamd2 libcholmod3 libcolamd2 libcxsparse3 libglpk-dev
  libglpk40 libgmp-dev libgmpxx4ldbl libgraphblas-dev libgraphblas6 libklu1 libldl2 libmetis5
  libmongoose2 librbio2 libsliplu1 libspqr2 libsuitesparse-dev libsuitesparseconfig5 libumfpack5
0 upgraded, 24 newly installed, 0 to remove and 49 not upgraded.
Need to get 23.7 MB of archives.
After this operation, 175 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.c

Now IPKnot is installed from source

In [7]:
!git clone https://github.com/satoken/ipknot.git
os.chdir("ipknot")
!mkdir build
os.chdir("build")
!cmake -DCMAKE_BUILD_TYPE=Release ..
!make
!make install # optional
os.chdir("/content")

fatal: destination path 'ipknot' already exists and is not an empty directory.
CMake Warning (dev) at CMakeLists.txt:6 (enable_language):
  project() should be called prior to this enable_language() call.
This warning is for project developers.  Use -Wno-dev to suppress it.

-- The CXX compiler identification is GNU 11.4.0
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- The C compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Found PkgConfig: /usr/bin/pkg-config (found version "1.8.0")
-- Checking for module 'RNAlib2'
--   Package 'RNAlib2', required by 'virtual:world', not found
-- Found GLPK: /usr/lib/x86_64-linux-gnu/libglpk.so
-- Conf

# S. cerevisiae (sce) 18 long non-coding RNA dataset
Data source: https://genie.weizmann.ac.il/pubs/PARS10/data/sce_genes_folded.tab.gz

In [ ]:
gh_path = "https://raw.githubusercontent.com/sinc-lab/lncRNA-folding/master/data/"
sce = pd.read_csv(gh_path + "sce_genes_folded.tab", delimiter='\t',
                  header=None, index_col=0,
                  names=("Gene ID", "sequence", "PARS-assisted folding"))

In [ ]:
# Sequences to process
sce18 = ["snR81", "snR34", "snR43", "snR44",  "snR31",  "snR10",
         "snR63", "snR11", "snR82", "snR17b", "snR17a", "snR37",
         "SCR1",  "SRG1",  "snR19", "snR30",  "LSR1",   "TLC1"]

# Compute structures

In [ ]:
def run_folding(fasta_name):

  # Compute structure
  os.system("ipknot -e lpc tmp.fasta > ipknot_tmp.fasta")

  # Clean output
  os.system("sed  -i '/Long-step dual simplex will be used/d' ./ipknot_tmp.fasta")

  return "ipknot_tmp.fasta"

In [ ]:
out_fasta_name = method_name + "_yeast18"
if os.path.exists(out_fasta_name + ".fasta"): os.remove(out_fasta_name + ".fasta")

print("   \t lnc \t len \t time")
for i, lnc in enumerate(sce18):

  start_time = time.time()
  seq = sce.loc[lnc]["sequence"]
  print(f"{i+1}/{len(sce18)}\t{lnc} \t {len(seq)}", end='\t')

  # Write a one-sequence fasta
  with open("tmp.fasta", "w") as ofile:
    ofile.write(f">{lnc}\n{seq}\n")

  dot_file_name = run_folding("tmp.fasta")

  # Concatenate outputs
  os.system("cat " + dot_file_name + " >> " + out_fasta_name + ".fasta")

  print(f"{time.time() - start_time: .1f} s")

   	 lnc 	 len 	 time
1/18	snR81 	 201	 0.3 s
2/18	snR34 	 203	 0.3 s
3/18	snR43 	 209	 0.3 s
4/18	snR44 	 211	 0.3 s
5/18	snR31 	 225	 0.4 s
6/18	snR10 	 245	 0.5 s
7/18	snR63 	 255	 0.6 s
8/18	snR11 	 258	 1.0 s
9/18	snR82 	 268	 0.5 s
10/18	snR17b 	 332	 1.2 s
11/18	snR17a 	 333	 1.2 s
12/18	snR37 	 386	 1.0 s
13/18	SCR1 	 522	 10.7 s
14/18	SRG1 	 551	 3.8 s
15/18	snR19 	 568	 2.7 s
16/18	snR30 	 606	 2.9 s
17/18	LSR1 	 1175	 6.7 s
18/18	TLC1 	 1301	 184.8 s
